In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ftx

In [2]:
df = pd.read_csv('trades_1.csv')
df = df.iloc[::-1]
df.head(0)

,ID,Time,Market,Side,Order Type,Size,Price,Total,Fee,Fee Currency


In [3]:
df_deposits = pd.read_csv('deposits.csv')
df_deposits = df_deposits.iloc[::-1]
df_deposits.head(0)

,,Time,Coin,Amount,Status,Additional info,Transaction ID,.1


In [4]:
df_withdrawals = pd.read_csv('withdrawals.csv')
df_withdrawals = df_withdrawals.iloc[::-1]
df_withdrawals.head(0)

,Time,Coin,Amount,Destination,Status,Transaction ID,fee,


In [5]:
wallet = {}

for index, row in df.iterrows():
    # spot
    if '/' in row.Market:
        asset, currency = row.Market.split('/')

        if currency not in wallet.keys():
            wallet[currency] = 0
        if asset not in wallet.keys():
            wallet[asset] = 0

        if row.Side == 'buy':
            wallet[currency] -= row.Total
            wallet[asset] += row.Size
        elif row.Side == 'sell':
            wallet[currency] += row.Total
            wallet[asset] -= row.Size
        else:
            raise ValueError

        wallet[row['Fee Currency']] -= row.Fee
    # derivative
    elif '-' in row.Market:
        assert(row['Fee Currency'] == 'USD')
        
        if row.Market not in wallet.keys():
            wallet[row.Market] = 0
            
        if row.Side == 'buy':
            wallet['USD'] -= row.Total
            wallet[row.Market] += row.Size
        elif row.Side == 'sell':
            wallet['USD'] += row.Total
            wallet[row.Market] -= row.Size
        else:
            raise ValueError
            
        wallet[row['Fee Currency']] -= row.Fee
    else:
        print("ERROR:")
        print(row)
        raise NotImplementedError
        

In [6]:
wallet_derivative = {'USD' : 0}

for index, row in df.iterrows():
    # derivative
    if '-' in row.Market:
        if row.Market not in wallet_derivative.keys():
            wallet_derivative[row.Market] = 0

        assert(row['Fee Currency'] == 'USD')
            
        if row.Side == 'buy':
            wallet_derivative['USD'] -= row.Total
            wallet_derivative[row.Market] += row.Size
        elif row.Side == 'sell':
            wallet_derivative['USD'] += row.Total
            wallet_derivative[row.Market] -= row.Size
        else:
            raise ValueError
            
        wallet_derivative[row['Fee Currency']] -= row.Fee


In [7]:
wallet_fee = {}

for index, row in df.iterrows():
    # spot
    if '/' in row.Market:
        asset, currency = row.Market.split('/')

        if currency not in wallet_fee.keys():
            wallet_fee[currency] = 0
        if asset not in wallet_fee.keys():
            wallet_fee[asset] = 0

        wallet_fee[row['Fee Currency']] -= row.Fee
    # derivative
    elif '-' in row.Market:
        assert(row['Fee Currency'] == 'USD')
        
        if row.Market not in wallet_fee.keys():
            wallet_fee[row.Market] = 0
            
        wallet_fee[row['Fee Currency']] -= row.Fee
    else:
        print("ERROR:")
        print(row)
        raise NotImplementedError
        

In [8]:
wallet_out = {}

for index, row in df_deposits.iterrows():
    if row.Coin not in wallet.keys():
        wallet[row.Coin] = 0
    if row.Coin not in wallet_out.keys():
        wallet_out[row.Coin] = 0
        
    wallet[row.Coin] += row.Amount
    wallet_out[row.Coin] -= row.Amount
    
    
for index, row in df_withdrawals.iterrows():
    if row.Coin not in wallet.keys():
        wallet[row.Coin] = 0
    if row.Coin not in wallet_out.keys():
        wallet_out[row.Coin] = 0
        
    wallet[row.Coin] -= row.Amount
    wallet_out[row.Coin] += row.Amount - row.fee

In [9]:
epsilon = 1e-4
wallet = {key:val for key, val in wallet.items() if val > epsilon or val < -epsilon}
wallet_derivative = {key:val for key, val in wallet_derivative.items() if val > epsilon or val < -epsilon}
wallet_fee = {key:val for key, val in wallet_fee.items() if val > epsilon or val < -epsilon}
wallet_out = {key:val for key, val in wallet_out.items() if val > epsilon or val < -epsilon}

In [10]:
wallet

{'USDT': -0.03696568018176549,
 'AAVE': 0.9977752944999994,
 'BNB': 0.9064604055000008,
 'ETH': -0.00021970874999974188,
 'FTT': 61.80568018600802,
 'LINK': 3.1977375550000042,
 'RUNE': -0.05097337499999721,
 'SOL': 4.957125663212599,
 'USD': -7186.629114309308,
 'MATIC': 99.92304596000001,
 'DOGE': 362.0,
 'XRP': 7.709141810000034,
 'EUR': 1300.0000000095606,
 'CEL': -0.00036904999999887167,
 'ATOM-PERP': 149.47,
 'BTC-PERP': 0.1,
 'AR-PERP': 102.70000000000002,
 'C98-PERP': 55.0,
 '1INCH': -0.17808443000000018,
 'GRT': 384.0,
 'RAY': 5.0,
 'AVAX': 2.2998609599999997,
 'KNC': -0.12871024000000142,
 'FTM': -0.028182269999998334,
 'USO': 0.00018645000000000814,
 'LUNC': -41743.76898033498,
 'ATOM': -0.00131988,
 'COIN': 0.5,
 'LOOKS-PERP': 1952.0,
 'APE': 7.4,
 'GMT-PERP': 871.0,
 'BTC-MOVE-WK-0617': -0.1513,
 'BTC-MOVE-0611': -0.41140000000000004}

In [11]:
wallet_out

{'USDT': -2482.879264050001,
 'EUR': -9770.0,
 'ETH': 1.8640342900000002,
 'BTC': -0.0003985,
 'XRP': 4.570655500000015,
 'MNGO': -0.01579000000000974,
 'BNB': 2.4608411500000007,
 'STARS': 0.3686519999999973,
 'SOL': 6.26,
 'ALCX': 0.338,
 'AVAX': 0.2}

In [12]:
wallet_derivative

{'USD': -8460.119973489147,
 'ATOM-PERP': 149.47,
 'BTC-PERP': 0.1,
 'AR-PERP': 102.70000000000002,
 'C98-PERP': 55.0,
 'LOOKS-PERP': 1952.0,
 'GMT-PERP': 871.0,
 'BTC-MOVE-WK-0617': -0.1513,
 'BTC-MOVE-0611': -0.41140000000000004}

In [13]:
wallet_fee

{'USDT': -0.829897157535,
 'AAVE': -0.0026490655,
 'BNB': -0.0020340545,
 'ETH': -0.00192804875,
 'FTT': -0.017202315,
 'LINK': -0.018715094999999998,
 'RUNE': -0.053673385,
 'SOL': -0.01209369,
 'TRX': -12.911670399999998,
 'USD': -477.00525377746607,
 'MATIC': -0.24224050000000003,
 'XAUT': -0.0012065706800000004,
 'ENJ': -0.29378940000000003,
 'DOGE': -0.206416,
 'TSLA': -0.00013965,
 'XRP': -0.8728191500000001,
 'UNI': -0.006902035,
 'CEL': -0.000836,
 'AKRO': -0.290966,
 'DYDX': -0.00019,
 'C98': -0.002888,
 'MNGO': -0.030685,
 'LRC': -0.005415,
 'FTM': -0.0034295,
 'LUNC': -0.366308505}

In [14]:
client = ftx.FtxClient()

# client.get_trades('BTC/USD', 2, datetime.datetime(2021, 1, 1).timestamp(), datetime.datetime(2021, 1, 2).timestamp())

def eval_wallet(w):
    total = 0
    for k in w.keys():
        if '-' in k:
            orderbook = client.get_orderbook(k, 1)
            value = np.mean(np.concatenate([np.array(orderbook['bids'])[:, 0], np.array(orderbook['asks'])[:, 0]]))
        elif k != 'USD':
            orderbook = client.get_orderbook(f'{k}/USD', 1)
            if orderbook['bids'] == [] or orderbook['asks'] == []:
                value = 0
            else:
                value = np.mean(np.concatenate([np.array(orderbook['bids'])[:, 0], np.array(orderbook['asks'])[:, 0]]))
        else:
            value = 1.0

        total += w[k] * value
    return total

In [15]:
value_wallet = eval_wallet(wallet)
value_wallet_derivative = eval_wallet(wallet_derivative)
value_wallet_fee = eval_wallet(wallet_fee)
value_wallet_out = eval_wallet(wallet_out)

In [16]:
results = value_wallet + value_wallet_out
print("wallet value".ljust(32), ':', round(value_wallet, 2))
print("wallet 'out' value".ljust(32), ':', round(value_wallet_out, 2))
print("results".ljust(32), ':', round(results, 2))
print("fee sum".ljust(32), ':', round(value_wallet_fee, 2))
print("derivative".ljust(32), ':', round((1-((results - value_wallet_derivative) / results))*100, 2), '%')

wallet value                     : 2370.6
wallet 'out' value               : -9022.76
results                          : -6652.16
fee sum                          : -486.69
derivative                       : 40.66 %
